# Train a Simple Audio Recognition Model

This notebook demonstrates how to train a 20 kB [Simple Audio Recognition](https://www.tensorflow.org/tutorials/sequences/audio_recognition) model to recognize keywords in speech.

The model created in this notebook is used in the [micro_speech](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/lite/micro/examples/micro_speech) example for [TensorFlow Lite for MicroControllers](https://www.tensorflow.org/lite/microcontrollers/overview).

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/micro/examples/micro_speech/train/train_micro_speech_model.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/micro/examples/micro_speech/train/train_micro_speech_model.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>


**Training is much faster using GPU acceleration.** Before you proceed, ensure you are using a GPU runtime by going to **Runtime -> Change runtime type** and set **Hardware accelerator: GPU**. Training 15,000 iterations will take 1.5 - 2 hours on a GPU runtime.

## Configure Defaults

**MODIFY** the following constants for your specific use case.

In [14]:
# A comma-delimited list of the words you want to train for.
# The options are: yes,no,up,down,left,right,on,off,stop,go
# All the other words will be used to train an "unknown" label and silent
# audio data with no spoken words will be used to train a "silence" label.
WANTED_WORDS = "snoring,off"

# The number of steps and learning rates can be specified as comma-separated
# lists to define the rate at each stage. For example,
# TRAINING_STEPS=12000,3000 and LEARNING_RATE=0.001,0.0001
# will run 12,000 training loops in total, with a rate of 0.001 for the first
# 8,000, and 0.0001 for the final 3,000.
# TRAINING_STEPS = "25,25"
# LEARNING_RATE = "0.005,0.005"

# Adriana
TRAINING_STEPS = "12000,3000"
LEARNING_RATE = "0.001,0.0001"

# Calculate the total number of steps, which is used to identify the checkpoint
# file name.
TOTAL_STEPS = str(sum(map(lambda string: int(string), TRAINING_STEPS.split(","))))

# Print the configuration to confirm it
print("Training these words: %s" % WANTED_WORDS)
print("Training steps in each stage: %s" % TRAINING_STEPS)
print("Learning rate in each stage: %s" % LEARNING_RATE)
print("Total number of training steps: %s" % TOTAL_STEPS)


Training these words: snoring,off
Training steps in each stage: 12000,3000
Learning rate in each stage: 0.001,0.0001
Total number of training steps: 15000


In [15]:
# Check version of python using
# https://medium.com/@nrk25693/how-to-add-your-conda-environment-to-your-jupyter-notebook-in-just-4-steps-abeab8b8d084
import sys
print(sys.executable)

/Users/ariadnarotaru/opt/anaconda3/bin/python


**DO NOT MODIFY** the following constants as they include filepaths used in this notebook and data that is shared during training and inference.

In [16]:
number_of_labels = WANTED_WORDS.count(',') + 1
number_of_total_labels = number_of_labels + 2 # for 'silence' and 'unknown' label
equal_percentage_of_training_samples = int(100.0/(number_of_total_labels))
SILENT_PERCENTAGE = equal_percentage_of_training_samples
UNKNOWN_PERCENTAGE = equal_percentage_of_training_samples

# Constants which are shared during training and inference
PREPROCESS = 'micro' # Other options 'micro', 'average', 'mfcc'
WINDOW_STRIDE = 20
MODEL_ARCHITECTURE = 'conv' # Other options include: single_fc, conv, tiny_conv,
                      # low_latency_conv, low_latency_svdf, tiny_embedding_conv

# # Constants used during training only
# VERBOSITY = 'DEBUG' # WARN, DEBUG
# EVAL_STEP_INTERVAL = '5'
# SAVE_STEP_INTERVAL = '25'

# Adriana
VERBOSITY = 'DEBUG' # WARN, DEBUG
EVAL_STEP_INTERVAL = '1000'
SAVE_STEP_INTERVAL = '1000'

# Constants for training directories and filepaths
DATASET_DIR = '/Users/ariadnarotaru/Desktop/249r/Snoring-Detection/Snoring_Dataset_@16000'
LOGS_DIR = 'logs/'
TRAIN_DIR = 'train/' # for training checkpoints and other files.
import os
if not os.path.exists(LOGS_DIR):
    os.mkdir(LOGS_DIR)
if not os.path.exists(TRAIN_DIR):
    os.mkdir(TRAIN_DIR)

# Constants for inference directories and filepaths
MODELS_DIR = 'models'
if not os.path.exists(MODELS_DIR):
  os.mkdir(MODELS_DIR)
MODEL_TF = os.path.join(MODELS_DIR, 'model.pb')
MODEL_TFLITE = os.path.join(MODELS_DIR, 'model.tflite')
FLOAT_MODEL_TFLITE = os.path.join(MODELS_DIR, 'float_model.tflite')
MODEL_TFLITE_MICRO = os.path.join(MODELS_DIR, 'model.cc')
SAVED_MODEL = os.path.join(MODELS_DIR, 'saved_model')

QUANT_INPUT_MIN = 0.0
QUANT_INPUT_MAX = 26.0
QUANT_INPUT_RANGE = QUANT_INPUT_MAX - QUANT_INPUT_MIN


## Setup Environment

Install Dependencies

In [17]:
#%tensorflow_version 1.x
import tensorflow as tf

**DELETE** any old data from previous runs


In [18]:
#!rm -rf {LOGS_DIR} {TRAIN_DIR} {MODELS_DIR}

Clone the TensorFlow Github Repository, which contains the relevant code required to run this tutorial.

In [19]:
#!git clone -q --depth 1 https://github.com/tensorflow/tensorflow

Load TensorBoard to visualize the accuracy and loss as training proceeds.


In [20]:
%load_ext tensorboard
%tensorboard --logdir {LOGS_DIR}

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6009 (pid 91738), started 16:44:22 ago. (Use '!kill 91738' to kill it.)

## Training

The following script downloads the dataset and begin training.

In [ ]:
!python tensorflow1/tensorflow/examples/speech_commands/train.py \
--data_dir={DATASET_DIR} \
--data_url='' \
--wanted_words={WANTED_WORDS} \
--preprocess={PREPROCESS} \
--silence_percentage={SILENT_PERCENTAGE} \
--unknown_percentage={UNKNOWN_PERCENTAGE} \
--window_stride={WINDOW_STRIDE} \
--model_architecture={MODEL_ARCHITECTURE} \
--how_many_training_steps={TRAINING_STEPS} \
--learning_rate={LEARNING_RATE} \
--train_dir={TRAIN_DIR} \
--summaries_dir={LOGS_DIR} \
--verbosity={VERBOSITY} \
--eval_step_interval={EVAL_STEP_INTERVAL} \
--save_step_interval={SAVE_STEP_INTERVAL} \
--dropout=0 \
--optimizer='momentum'


2020-12-12 13:50:07.285092: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-12-12 13:50:07.302745: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fa624000330 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-12-12 13:50:07.302781: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1212 13:50:07.892526 4705514944 deprecation.py:323] From /Users/ariadnarotaru/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/losses/losses_impl.py:121: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Training from step: 1 
I1

DEBUG:tensorflow:Step #37: rate 0.001000, accuracy 67.0%, cross entropy 0.878218
I1212 13:50:52.474992 4705514944 train.py:245] Step #37: rate 0.001000, accuracy 67.0%, cross entropy 0.878218
DEBUG:tensorflow:Step #38: rate 0.001000, accuracy 64.0%, cross entropy 0.934827
I1212 13:50:53.596431 4705514944 train.py:245] Step #38: rate 0.001000, accuracy 64.0%, cross entropy 0.934827
DEBUG:tensorflow:Step #39: rate 0.001000, accuracy 56.0%, cross entropy 0.924412
I1212 13:50:54.701964 4705514944 train.py:245] Step #39: rate 0.001000, accuracy 56.0%, cross entropy 0.924412
DEBUG:tensorflow:Step #40: rate 0.001000, accuracy 59.0%, cross entropy 0.856038
I1212 13:50:55.796903 4705514944 train.py:245] Step #40: rate 0.001000, accuracy 59.0%, cross entropy 0.856038
DEBUG:tensorflow:Step #41: rate 0.001000, accuracy 59.0%, cross entropy 0.821372
I1212 13:50:56.995512 4705514944 train.py:245] Step #41: rate 0.001000, accuracy 59.0%, cross entropy 0.821372
DEBUG:tensorflow:Step #42: rate 0.001000

DEBUG:tensorflow:Step #80: rate 0.001000, accuracy 76.0%, cross entropy 0.704317
I1212 13:51:42.118426 4705514944 train.py:245] Step #80: rate 0.001000, accuracy 76.0%, cross entropy 0.704317
DEBUG:tensorflow:Step #81: rate 0.001000, accuracy 72.0%, cross entropy 0.683217
I1212 13:51:43.326251 4705514944 train.py:245] Step #81: rate 0.001000, accuracy 72.0%, cross entropy 0.683217
DEBUG:tensorflow:Step #82: rate 0.001000, accuracy 70.0%, cross entropy 0.736609
I1212 13:51:44.694478 4705514944 train.py:245] Step #82: rate 0.001000, accuracy 70.0%, cross entropy 0.736609
DEBUG:tensorflow:Step #83: rate 0.001000, accuracy 80.0%, cross entropy 0.645841
I1212 13:51:45.951815 4705514944 train.py:245] Step #83: rate 0.001000, accuracy 80.0%, cross entropy 0.645841
DEBUG:tensorflow:Step #84: rate 0.001000, accuracy 71.0%, cross entropy 0.774348
I1212 13:51:47.310782 4705514944 train.py:245] Step #84: rate 0.001000, accuracy 71.0%, cross entropy 0.774348
DEBUG:tensorflow:Step #85: rate 0.001000

DEBUG:tensorflow:Step #123: rate 0.001000, accuracy 81.0%, cross entropy 0.504543
I1212 13:52:37.179898 4705514944 train.py:245] Step #123: rate 0.001000, accuracy 81.0%, cross entropy 0.504543
DEBUG:tensorflow:Step #124: rate 0.001000, accuracy 79.0%, cross entropy 0.581685
I1212 13:52:38.518228 4705514944 train.py:245] Step #124: rate 0.001000, accuracy 79.0%, cross entropy 0.581685
DEBUG:tensorflow:Step #125: rate 0.001000, accuracy 86.0%, cross entropy 0.399797
I1212 13:52:39.657389 4705514944 train.py:245] Step #125: rate 0.001000, accuracy 86.0%, cross entropy 0.399797
DEBUG:tensorflow:Step #126: rate 0.001000, accuracy 73.0%, cross entropy 0.581161
I1212 13:52:40.965111 4705514944 train.py:245] Step #126: rate 0.001000, accuracy 73.0%, cross entropy 0.581161
DEBUG:tensorflow:Step #127: rate 0.001000, accuracy 80.0%, cross entropy 0.514124
I1212 13:52:42.080121 4705514944 train.py:245] Step #127: rate 0.001000, accuracy 80.0%, cross entropy 0.514124
DEBUG:tensorflow:Step #128: ra

DEBUG:tensorflow:Step #166: rate 0.001000, accuracy 80.0%, cross entropy 0.540846
I1212 13:53:31.161050 4705514944 train.py:245] Step #166: rate 0.001000, accuracy 80.0%, cross entropy 0.540846
DEBUG:tensorflow:Step #167: rate 0.001000, accuracy 83.0%, cross entropy 0.442050
I1212 13:53:32.269838 4705514944 train.py:245] Step #167: rate 0.001000, accuracy 83.0%, cross entropy 0.442050
DEBUG:tensorflow:Step #168: rate 0.001000, accuracy 74.0%, cross entropy 0.567879
I1212 13:53:33.591351 4705514944 train.py:245] Step #168: rate 0.001000, accuracy 74.0%, cross entropy 0.567879
DEBUG:tensorflow:Step #169: rate 0.001000, accuracy 86.0%, cross entropy 0.338866
I1212 13:53:34.966320 4705514944 train.py:245] Step #169: rate 0.001000, accuracy 86.0%, cross entropy 0.338866
DEBUG:tensorflow:Step #170: rate 0.001000, accuracy 89.0%, cross entropy 0.337285
I1212 13:53:36.165882 4705514944 train.py:245] Step #170: rate 0.001000, accuracy 89.0%, cross entropy 0.337285
DEBUG:tensorflow:Step #171: ra

...

DEBUG:tensorflow:Step #6548: rate 0.001000, accuracy 100.0%, cross entropy 0.021240
I1212 16:05:57.519537 4705514944 train.py:245] Step #6548: rate 0.001000, accuracy 100.0%, cross entropy 0.021240
DEBUG:tensorflow:Step #6549: rate 0.001000, accuracy 99.0%, cross entropy 0.022847
I1212 16:05:58.648864 4705514944 train.py:245] Step #6549: rate 0.001000, accuracy 99.0%, cross entropy 0.022847
DEBUG:tensorflow:Step #6550: rate 0.001000, accuracy 100.0%, cross entropy 0.014004
I1212 16:05:59.794007 4705514944 train.py:245] Step #6550: rate 0.001000, accuracy 100.0%, cross entropy 0.014004
DEBUG:tensorflow:Step #6551: rate 0.001000, accuracy 99.0%, cross entropy 0.063075
I1212 16:06:00.938905 4705514944 train.py:245] Step #6551: rate 0.001000, accuracy 99.0%, cross entropy 0.063075
DEBUG:tensorflow:Step #6552: rate 0.001000, accuracy 98.0%, cross entropy 0.071411
I1212 16:06:02.116614 4705514944 train.py:245] Step #6552: rate 0.001000, accuracy 98.0%, cross entropy 0.071411
DEBUG:tensorflow

DEBUG:tensorflow:Step #6590: rate 0.001000, accuracy 98.0%, cross entropy 0.030368
I1212 16:06:46.320134 4705514944 train.py:245] Step #6590: rate 0.001000, accuracy 98.0%, cross entropy 0.030368
DEBUG:tensorflow:Step #6591: rate 0.001000, accuracy 97.0%, cross entropy 0.102578
I1212 16:06:47.485957 4705514944 train.py:245] Step #6591: rate 0.001000, accuracy 97.0%, cross entropy 0.102578
DEBUG:tensorflow:Step #6592: rate 0.001000, accuracy 100.0%, cross entropy 0.005415
I1212 16:06:48.540940 4705514944 train.py:245] Step #6592: rate 0.001000, accuracy 100.0%, cross entropy 0.005415
DEBUG:tensorflow:Step #6593: rate 0.001000, accuracy 99.0%, cross entropy 0.028013
I1212 16:06:49.581556 4705514944 train.py:245] Step #6593: rate 0.001000, accuracy 99.0%, cross entropy 0.028013
DEBUG:tensorflow:Step #6594: rate 0.001000, accuracy 97.0%, cross entropy 0.062057
I1212 16:06:50.721894 4705514944 train.py:245] Step #6594: rate 0.001000, accuracy 97.0%, cross entropy 0.062057
DEBUG:tensorflow:S

DEBUG:tensorflow:Step #6632: rate 0.001000, accuracy 98.0%, cross entropy 0.051533
I1212 16:07:39.723458 4705514944 train.py:245] Step #6632: rate 0.001000, accuracy 98.0%, cross entropy 0.051533
DEBUG:tensorflow:Step #6633: rate 0.001000, accuracy 99.0%, cross entropy 0.057169
I1212 16:07:40.869976 4705514944 train.py:245] Step #6633: rate 0.001000, accuracy 99.0%, cross entropy 0.057169
DEBUG:tensorflow:Step #6634: rate 0.001000, accuracy 97.0%, cross entropy 0.067568
I1212 16:07:42.217163 4705514944 train.py:245] Step #6634: rate 0.001000, accuracy 97.0%, cross entropy 0.067568
DEBUG:tensorflow:Step #6635: rate 0.001000, accuracy 99.0%, cross entropy 0.055231
I1212 16:07:43.380197 4705514944 train.py:245] Step #6635: rate 0.001000, accuracy 99.0%, cross entropy 0.055231
DEBUG:tensorflow:Step #6636: rate 0.001000, accuracy 100.0%, cross entropy 0.009197
I1212 16:07:44.468142 4705514944 train.py:245] Step #6636: rate 0.001000, accuracy 100.0%, cross entropy 0.009197
DEBUG:tensorflow:S

DEBUG:tensorflow:Step #6674: rate 0.001000, accuracy 98.0%, cross entropy 0.034316
I1212 16:08:28.716639 4705514944 train.py:245] Step #6674: rate 0.001000, accuracy 98.0%, cross entropy 0.034316
DEBUG:tensorflow:Step #6675: rate 0.001000, accuracy 99.0%, cross entropy 0.024588
I1212 16:08:29.734264 4705514944 train.py:245] Step #6675: rate 0.001000, accuracy 99.0%, cross entropy 0.024588
DEBUG:tensorflow:Step #6676: rate 0.001000, accuracy 99.0%, cross entropy 0.021874
I1212 16:08:30.779773 4705514944 train.py:245] Step #6676: rate 0.001000, accuracy 99.0%, cross entropy 0.021874
DEBUG:tensorflow:Step #6677: rate 0.001000, accuracy 100.0%, cross entropy 0.003249
I1212 16:08:31.827642 4705514944 train.py:245] Step #6677: rate 0.001000, accuracy 100.0%, cross entropy 0.003249
DEBUG:tensorflow:Step #6678: rate 0.001000, accuracy 98.0%, cross entropy 0.053087
I1212 16:08:32.933752 4705514944 train.py:245] Step #6678: rate 0.001000, accuracy 98.0%, cross entropy 0.053087
DEBUG:tensorflow:S

DEBUG:tensorflow:Step #6716: rate 0.001000, accuracy 99.0%, cross entropy 0.067493
I1212 16:09:15.460736 4705514944 train.py:245] Step #6716: rate 0.001000, accuracy 99.0%, cross entropy 0.067493
DEBUG:tensorflow:Step #6717: rate 0.001000, accuracy 99.0%, cross entropy 0.038341
I1212 16:09:16.550486 4705514944 train.py:245] Step #6717: rate 0.001000, accuracy 99.0%, cross entropy 0.038341
DEBUG:tensorflow:Step #6718: rate 0.001000, accuracy 97.0%, cross entropy 0.123702
I1212 16:09:17.641175 4705514944 train.py:245] Step #6718: rate 0.001000, accuracy 97.0%, cross entropy 0.123702
DEBUG:tensorflow:Step #6719: rate 0.001000, accuracy 99.0%, cross entropy 0.033746
I1212 16:09:18.747756 4705514944 train.py:245] Step #6719: rate 0.001000, accuracy 99.0%, cross entropy 0.033746
DEBUG:tensorflow:Step #6720: rate 0.001000, accuracy 98.0%, cross entropy 0.030199
I1212 16:09:19.830511 4705514944 train.py:245] Step #6720: rate 0.001000, accuracy 98.0%, cross entropy 0.030199
DEBUG:tensorflow:Ste

## Generate a TensorFlow Model for Inference

Combine relevant training results (graph, weights, etc) into a single file for inference. This process is known as freezing a model and the resulting model is known as a frozen model/graph, as it cannot be further re-trained after this process.

In [ ]:
#!rm -rf {SAVED_MODEL}
!python tensorflow1/tensorflow/examples/speech_commands/freeze.py \
--wanted_words=$WANTED_WORDS \
--window_stride_ms=$WINDOW_STRIDE \
--preprocess=$PREPROCESS \
--model_architecture=$MODEL_ARCHITECTURE \
--start_checkpoint=$TRAIN_DIR$MODEL_ARCHITECTURE'.ckpt-'{TOTAL_STEPS} \
--save_format=saved_model \
--output_file={SAVED_MODEL}


## Generate a TensorFlow Lite Model

Convert the frozen graph into a TensorFlow Lite model, which is fully quantized for use with embedded devices.

The following cell will also print the model size, which will be under 20 kilobytes.

In [ ]:
import sys
# We add this path so we can import the speech processing modules.
sys.path.append("tensorflow1/tensorflow/examples/speech_commands/")
import input_data
import models
import numpy as np

In [ ]:
SAMPLE_RATE = 16000
CLIP_DURATION_MS = 1000
WINDOW_SIZE_MS = 30.0
FEATURE_BIN_COUNT = 40
# BACKGROUND_FREQUENCY = 0
# BACKGROUND_VOLUME_RANGE = 0
# Adriana:
BACKGROUND_FREQUENCY = 0.8
BACKGROUND_VOLUME_RANGE = 0.1
TIME_SHIFT_MS = 100.0

DATA_URL = '' #'https://storage.googleapis.com/download.tensorflow.org/data/speech_commands_v0.02.tar.gz'
VALIDATION_PERCENTAGE = 10
TESTING_PERCENTAGE = 10


In [ ]:
model_settings = models.prepare_model_settings(
    len(input_data.prepare_words_list(WANTED_WORDS.split(','))),
    SAMPLE_RATE, CLIP_DURATION_MS, WINDOW_SIZE_MS,
    WINDOW_STRIDE, FEATURE_BIN_COUNT, PREPROCESS)
# audio_processor = input_data.AudioProcessor(
#     DATA_URL, DATASET_DIR,
#     WANTED_WORDS.split(','), VALIDATION_PERCENTAGE,
#     TESTING_PERCENTAGE, model_settings, LOGS_DIR)

# # Adriana:
audio_processor = input_data.AudioProcessor(
    DATA_URL, DATASET_DIR,
    SILENT_PERCENTAGE, UNKNOWN_PERCENTAGE,
    WANTED_WORDS.split(','), VALIDATION_PERCENTAGE,
    TESTING_PERCENTAGE, model_settings, LOGS_DIR)


In [ ]:
#with tf.Session() as sess:
with tf.compat.v1.Session() as sess:
  float_converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL)
  float_tflite_model = float_converter.convert()
  float_tflite_model_size = open(FLOAT_MODEL_TFLITE, "wb").write(float_tflite_model)
  print("Float model is %d bytes" % float_tflite_model_size)

  converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL)
  converter.optimizations = [tf.lite.Optimize.DEFAULT]
  converter.inference_input_type = tf.compat.v1.lite.constants.INT8 #tf.lite.constants.INT8
  converter.inference_output_type = tf.compat.v1.lite.constants.INT8 #tf.lite.constants.INT8
  def representative_dataset_gen():
    for i in range(100):
      print(i)
      data, _ = audio_processor.get_data(1, i*1, model_settings,
                                         BACKGROUND_FREQUENCY, 
                                         BACKGROUND_VOLUME_RANGE,
                                         TIME_SHIFT_MS,
                                         'testing',
                                         sess)

      flattened_data = np.array(data.flatten(), dtype=np.float32).reshape(1, 1960)
      yield [flattened_data]
  converter.representative_dataset = representative_dataset_gen
  tflite_model = converter.convert()
  tflite_model_size = open(MODEL_TFLITE, "wb").write(tflite_model)
  print("Quantized model is %d bytes" % tflite_model_size)


## Testing the TensorFlow Lite model's accuracy

Verify that the model we've exported is still accurate, using the TF Lite Python API and our test set.

In [ ]:
# Helper function to run inference
def run_tflite_inference(tflite_model_path, model_type="Float"):
  # Load test data
  np.random.seed(0) # set random seed for reproducible test results.
  #with tf.Session() as sess:
  with tf.compat.v1.Session() as sess:
    test_data, test_labels = audio_processor.get_data(
        -1, 0, model_settings, BACKGROUND_FREQUENCY, BACKGROUND_VOLUME_RANGE,
        TIME_SHIFT_MS, 'testing', sess)
  test_data = np.expand_dims(test_data, axis=1).astype(np.float32)
  print(" AUDIO PROCESSOR OUTPUT ", audio_processor.output_.shape)
  # Initialize the interpreter
  interpreter = tf.lite.Interpreter(tflite_model_path)
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  # For quantized models, manually quantize the input data from float to integer
  if model_type == "Quantized":
    input_scale, input_zero_point = input_details["quantization"]
    test_data = test_data / input_scale + input_zero_point
    test_data = test_data.astype(input_details["dtype"])

  correct_predictions = 0
  for i in range(len(test_data)):
    interpreter.set_tensor(input_details["index"], test_data[i])
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]
    top_prediction = output.argmax()
    correct_predictions += (top_prediction == test_labels[i])

  print('%s model accuracy is %f%% (Number of test samples=%d)' % (
      model_type, (correct_predictions * 100) / len(test_data), len(test_data)))

In [ ]:
# Compute float model accuracy
run_tflite_inference(FLOAT_MODEL_TFLITE)

# Compute quantized model accuracy
run_tflite_inference(MODEL_TFLITE, model_type='Quantized')

## Generate a TensorFlow Lite for MicroControllers Model
Convert the TensorFlow Lite model into a C source file that can be loaded by TensorFlow Lite for Microcontrollers.

In [ ]:
# Install xxd if it is not available

# first install xxd
# !apt-get update && apt-get -qq install xxd

# Convert to a C source file
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}

# Update variable names
#REPLACE_TEXT = MODEL_TFLITE.replace('/', '_').replace('.', '_')
#!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_TFLITE_MICRO}

In [ ]:
# print(REPLACE_TEXT)
print(MODEL_TFLITE_MICRO)
print(MODEL_TFLITE)

## Deploy to a Microcontroller

Follow the instructions in the [micro_speech](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/lite/micro/examples/micro_speech) README.md for [TensorFlow Lite for MicroControllers](https://www.tensorflow.org/lite/microcontrollers/overview) to deploy this model on a specific microcontroller.

**Reference Model:** If you have not modified this notebook, you can follow the instructions as is, to deploy the model. Refer to the [`micro_speech/train/models`](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/micro/examples/micro_speech/train/models) directory to access the models generated in this notebook.

**New Model:** If you have generated a new model to identify different words: (i) Update `kCategoryCount` and `kCategoryLabels` in [`micro_speech/micro_features/micro_model_settings.h`](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/micro/examples/micro_speech/micro_features/micro_model_settings.h) and (ii) Update the values assigned to the variables defined in [`micro_speech/micro_features/model.cc`](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/micro/examples/micro_speech/micro_features/model.cc) with values displayed after running the following cell.

In [ ]:
# Print the C source file
!cat {MODEL_TFLITE_MICRO}